# 2-1. Prompt Template

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# env 파일이 안들어와서 강제 로드
import os

# .env 파일 강제 로드
dotenv_path = os.path.join(os.getcwd(), ".env")
load_dotenv(dotenv_path)

True

In [3]:
# LangSmith 추적을 설정 https://smith.langchain.com
!pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름
logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [5]:
from dotenv import load_dotenv
import os

# ✅ .env 파일 로드
load_dotenv()

# ✅ 환경 변수 확인
api_key = os.getenv("LANGSMITH_API_KEY")
#print(f"LANGSMITH API Key: {api_key}")


### 방법 1. from_template() 메소드 사용하여 PromptTemplate 객체 생성

In [12]:
from langchain_core.prompts import PromptTemplate

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['movie'], template='{movie}의 주인공은 누구인가요?')

In [13]:
# country 변수에 값을 넣어서 문장을 생성할 수 있음
# prompt 생성 .format 메소드 이용하여 변수에 값 넣기
prompt = prompt.format(movie = "Avatar")
prompt

'Avatar의 주인공은 누구인가요?'

In [14]:
# template 정의
template = "{movie}의 주인공은 누구인가요?"

# from_template 메소드로 Prompt Template 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [15]:
# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke("Avatar").content

'영화 "아바타"의 주인공은 제이크 솔리입니다.'

### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성
- : 추가 유효성 검사를 위해 'input_variables'를 명시적으로 지정
- 이러한 변수는 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치하는 경우 예외를 발생시킴

In [16]:
# template 정의
template = "{movie}의 여자 주인공은 누구인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template = template,
    input_variables = ["movie"]
)

prompt

PromptTemplate(input_variables=['movie'], template='{movie}의 여자 주인공은 누구인가요?')

In [18]:
# prompt 생성
prompt.format(movie = "Avatar")

'Avatar의 여자 주인공은 누구인가요?'

In [23]:
# template 정의
template = "{movie1}와 {movie2}의 여자 주인공은 각각 누구인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template = template,
    input_variables = ["movie1"],
    partial_variables = {
        "movie2": "La la land" #dictionary 형태로 partial_variables 전달
    },
)

prompt

PromptTemplate(input_variables=['movie1'], partial_variables={'movie2': 'La la land'}, template='{movie1}와 {movie2}의 여자 주인공은 각각 누구인가요?')

In [24]:
prompt.format(movie1 = "Avatar")

'Avatar와 La la land의 여자 주인공은 각각 누구인가요?'

In [27]:
prompt_partial = prompt.partial(movie2 = "La la land")
prompt_partial

PromptTemplate(input_variables=['movie1'], partial_variables={'movie2': 'La la land'}, template='{movie1}와 {movie2}의 여자 주인공은 각각 누구인가요?')

In [28]:
prompt_partial.format(movie1 = "Avatar")

'Avatar와 La la land의 여자 주인공은 각각 누구인가요?'

- `PromptTemplate.format()`에서는 partial_variables가 고정됨
    - partial_variables로 설정된 값은 format()을 사용할 때 변경되지 않음
- 그러나 `chain.invoke()`에서는 llm이 추가적인 입력을 허용할 수 있음

In [29]:
chain = prompt_partial | llm

In [ ]:
chain.invoke("Avatar").content

'Avatar의 여자 주인공은 Neytiri이고, La La Land의 여자 주인공은 Mia이다.'

In [35]:
chain.invoke({"movie1": "Substance", "movie2": "Elemental"}) #틀림

AIMessage(content='Substance의 여자 주인공은 에밀리 (Emily)이고, Elemental의 여자 주인공은 제이드 (Jade)입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 30, 'total_tokens': 67, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2c431c77-c6e1-47c2-8e29-69f06652bc0f-0', usage_metadata={'input_tokens': 30, 'output_tokens': 37, 'total_tokens': 67})

In [34]:
# 한국 영화
chain.invoke({"movie1": "밀수", "movie2": "타짜"}) #틀림

AIMessage(content='밀수의 여자 주인공은 조수미이고, 타짜의 여자 주인공은 김하늘입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 34, 'total_tokens': 70, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dfcae6f5-1a6d-4247-ba0b-c494c0156eff-0', usage_metadata={'input_tokens': 34, 'output_tokens': 36, 'total_tokens': 70})

### hallucination 해결: RAG 사용*

In [44]:
!pip install wikipedia

In [47]:

from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

# Wikipedia 검색 API 래퍼 생성
wiki_api = WikipediaAPIWrapper()

# Wikipedia 검색 도구 생성
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)

# LLM + Wikipedia 검색 에이전트 설정
llm = ChatOpenAI(model="gpt-4o")

agent = initialize_agent(
    tools=[wiki_tool],
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
)

# 질문 실행
question = "영화 Substance와 영화 Elemental의 여자 주인공은 누구인가요?"
response = agent.run(question)

print(response)



> Entering new AgentExecutor chain...
Action: [wikipedia]
Action Input: Substance (film)
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: wikipedia
Action Input: Substance (film) female lead actress

Observation: Page: The Substance
Summary: The Substance is a 2024 body horror film written and directed by Coralie Fargeat. It follows a fading celebrity, Elisabeth Sparkle (Demi Moore) who, after being fired by her producer (Dennis Quaid) due to her age, uses a black market drug that creates a younger version of herself (Margaret Qualley) with unexpected side effects. It has been widely compared to Death Becomes Her (1992).
Noted for its satirical elements and grotesque, hyperrealistic imagery, The Substance is Fargeat's second feature film, after Revenge. Motivated by societal pressures on women's bodies and aging, she wrote the screenplay over two years, assembling a team to co-produce the film between France, the United Kingdom, and the United Sta

/opt/anaconda3/envs/langchain-env/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/anaconda3/envs/langchain-env/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Elemental (2023 film)
Summary: Elemental is a 2023 American animated romantic comedy-drama film produced by Pixar Animation Studios for Walt Disney Pictures. Directed by Peter Sohn and produced by Denise Ream, it was written by Sohn, John Hoberg, Kat Likkel, and Brenda Hsueh. The film stars the voices of Leah Lewis, Mamoudou Athie, Ronnie del Carmen, Shila Ommi, Wendi McLendon-Covey, and Catherine O'Hara. Set in a world inhabited by anthropomorphic elements of nature, the story follows fire element Ember Lumen and water element Wade Ripple, who spend time together in the city while trying to save a convenience store owned by Ember's father, Bernie.
Development of Elemental began when Sohn pitched the concept to Pixar based on the idea of whether fire and water could ever connect or not. The film draws inspiration from Sohn's youth, growing up as the son of immigrants in New York City during the 1970s, highlighting the city's distinct cultural and ethnic diversity wh

### 'partial_variables': 부분 변수 채움
- 항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우 ex) 날짜, 시간
- 항상 현재 날짜가 표시되기를 원하는 프롬프트가 있을 경우, 현재 날짜 반환 함수를 사용하여 프롬프트를 부분적으로 변경할 수 있으면 매우 편리함

In [53]:
from datetime import datetime, timedelta

# 오늘 날짜 출력
datetime.now().strftime("%B %d")

'February 24'

In [54]:
# 날짜 반환 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

# 어제 날짜 반환 함수 정의
def get_yesterday():
    return (datetime.now() - timedelta(days=1)).strftime("%B %d")

In [55]:
prompt = PromptTemplate(
    template = "어제의 날짜는 {yesterday} 입니다. 어제가 생일인 정치인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables = {"yesterday": get_yesterday}, #dictionary 형태로 partial_variables 전달
)

In [56]:
# prompt 생성
prompt.format(n=5)

'어제의 날짜는 February 23 입니다. 어제가 생일인 정치인 5명을 나열해 주세요. 생년월일을 표기해주세요.'

In [57]:
# chain 생성
chain = prompt | llm

In [ ]:
# chain 실행 후 결과 확인
print(chain.invoke(5).content) # 태어난 연도는 맞고 날짜는 틀림

February 23일이 생일인 정치인 중 잘 알려진 몇 명을 소개하겠습니다.

1. 빌리 브란트 (Willy Brandt) - 1913년 2월 23일: 독일의 정치인으로 서독의 총리를 역임했습니다.
2. 박근혜 (Park Geun-hye) - 1952년 2월 23일: 대한민국의 정치인으로 제18대 대통령을 지냈습니다.
3. 조지프 W. 마틴 주니어 (Joseph W. Martin Jr.) - 1884년 2월 23일: 미국의 정치인으로, 하원의장을 역임했습니다.
4. 빅터 야누코비치 (Viktor Yanukovych) - 1950년 2월 23일: 우크라이나의 정치인으로, 대통령을 역임했습니다.
5. 키라 라드릭 (Kira Rudik) - 1985년 2월 23일: 우크라이나의 정치인으로, 하원의원을 역임하고 있습니다.

이 외에도 여러 정치인들이 이 날짜에 태어났을 수 있으며, 생일을 기념하는 다른 인물들도 있을 수 있습니다.


In [60]:
print(chain.invoke({"yesterday": "Mar 02", "n": 3}).content)

어제 날짜가 3월 2일이라면, 그 날짜에 태어난 잘 알려진 정치인 중 세 명을 다음과 같이 나열할 수 있습니다.

1. 미하일 고르바초프 (Mikhail Gorbachev) - 1931년 3월 2일 출생. 소련의 마지막 지도자로, 페레스트로이카와 글라스노스트 정책을 추진했습니다.

2. 샘 휴스턴 (Sam Houston) - 1793년 3월 2일 출생. 미국의 정치인으로, 텍사스 공화국의 초대 대통령이자 미국 상원의원 및 텍사스 주지사를 역임했습니다.

3. 모리스 맥도널드 (Maurice Macdonald) - 1902년 3월 2일 출생. 미국의 정치인으로, 여러 정치적 역할을 수행했습니다.

이 정치인들은 3월 2일에 태어났으며, 각자의 분야에서 중요한 역할을 했습니다.


### 파일로부터 template 읽어오기

In [62]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/movie.yaml")
prompt

PromptTemplate(name='movie_info', input_variables=['movie'], template='{movie}의 정보에 대해서 알려주세요.\n영화의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 영화 제목\n2. 장르\n3. 감독\n4. 배우')

In [63]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/movie.yaml", encoding="utf-8")
prompt

PromptTemplate(name='movie_info', input_variables=['movie'], template='{movie}의 정보에 대해서 알려주세요.\n영화의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 영화 제목\n2. 장르\n3. 감독\n4. 배우')

In [64]:
prompt.format(movie="Avatar")

'Avatar의 정보에 대해서 알려주세요.\n영화의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 영화 제목\n2. 장르\n3. 감독\n4. 배우'

In [ ]:
prompt = load_prompt("prompts/movie.yaml")
print(prompt.format(movie="Avatar")) #출력 모양이 다름

Avatar의 정보에 대해서 알려주세요.
영화의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 영화 제목
2. 장르
3. 감독
4. 배우


### MessagePlaceholder

In [67]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name = "conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)

chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.')), MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='지금까지의 대화를 {word_count} 단어로 요약합니다.'))])

In [77]:
# conversation 대화 목록을 나중에 추가하고자 할 떄 MessagesPlaceholder 사용
formatted_chat_prompt = chat_prompt.format(
    word_count = 5,
    conversation = [
        ("human", "안녕하세요! 저는 도전자 쉐프 나폴리맛피아입니다."),
        ("ai", "반가워요! 나폴리맛피아님의 요리를 평가하겠습니다.")
    ],
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 도전자 쉐프 나폴리맛피아입니다.
AI: 반가워요! 나폴리맛피아님의 요리를 평가하겠습니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [78]:
# chain 생성
chain = chat_prompt | llm | StrOutputParser()

In [79]:
# chain 실행 및 확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            ("human", "안녕하세요! 저는 도전자 쉐프 나폴리맛피아입니다.",),
            ("ai", "반가워요! 나폴리맛피아님의 요리를 평가하겠습니다."),
        ],
    }
)

'도전자 쉐프, 나폴리맛피아 소개.'

# 2-2. 퓨샷 프롬프트

### FewShotPromptTemplate

In [85]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# 객체 생성
llm = ChatOpenAI(
    temperature = 0, # 창의성 (0: 보수적, 1: 창의적)
    model_name = "gpt-4-turbo" # 모델명
)

# 질의내용
question = "대한민국의 수도는 뭐야?"

# 질의
answer = llm.stream(question)
stream_response(answer)

대한민국의 수도는 서울입니다.

In [90]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# few-shot 예제 데이터 생성

examples = [
    {
        "question": "나폴리맛피아와 에드워드리 중 누가 요리 대결에서 이겼나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 나폴리맛피아는 몇 점을 받았나요?
중간 답변: 나폴리맛피아는 92점을 받았습니다.
추가 질문: 에드워드리는 몇 점을 받았나요?
중간 답변: 에드워드리는 88점을 받았습니다.
최종 답변은: 나폴리맛피아
""",
    },
    {
        "question": "카카오의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
        추가 질문: 카카오의 창립자는 누구인가요?
        중간 답변: 카카오는 김범수에 의해 창립되었습니다.
        추가 질문: 김범수는 언제 태어났나요?
        중간 답변: 김범수는 1966년 3월 8일에 태어났습니다.
        최종 답변은: 1966년 3월 8일"
    
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해에 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
"""
    },
    {
        
        "question": "살인의 추억과 마더의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 살인의 추억의 감독은 누구인가요?
중간 답변: 살인의 추억의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
추가 질문: 마더의 감독은 누구인가요?
중간 답변: 마더의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예"
    
""",
    },
]

In [91]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question:
나폴리맛피아와 에드워드리 중 누가 요리 대결에서 이겼나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 나폴리맛피아는 몇 점을 받았나요?
중간 답변: 나폴리맛피아는 92점을 받았습니다.
추가 질문: 에드워드리는 몇 점을 받았나요?
중간 답변: 에드워드리는 88점을 받았습니다.
최종 답변은: 나폴리맛피아



In [ ]:
## 방법1
prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix = "Question:\n{question}\nAnswer:",
    input_variables = ["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question = question)
print(final_prompt)

Question:
나폴리맛피아와 에드워드리 중 누가 요리 대결에서 이겼나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 나폴리맛피아는 몇 점을 받았나요?
중간 답변: 나폴리맛피아는 92점을 받았습니다.
추가 질문: 에드워드리는 몇 점을 받았나요?
중간 답변: 에드워드리는 88점을 받았습니다.
최종 답변은: 나폴리맛피아


Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
율곡 이이의 어머니가 태어난 해에 통치하던 왕은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군


Question:
올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


In [92]:
# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.
최종 답변은: 1998년에 Bill Gates의 나이는 43살입니다.

In [ ]:
## 방법2
prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix = "Question:\n{question}\nAnswer:",
    input_variables = ["question"],
)

#chain 생성
chain = prompt | llm | StrOutputParser()

# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.
최종 답변은: 1998년에 Bill Gates의 나이는 43살입니다.

**<방법 1, 방법 2 차이점>**
| 구분 | 방법1 (`format()` 후 실행) | 방법2 (`chain` 사용) |
|---|---|---|
| **프롬프트 생성** | `format()`을 직접 호출해야 함 | `chain.invoke()` 또는 `chain.stream()`이 자동 처리 |
| **실행 방식** | 수동으로 `final_prompt`를 만든 후 `llm.stream()` 실행 | `chain`을 통해 `FewShotPromptTemplate → LLM → Output Parsing`을 자동화 |
| **출력 처리** | `llm.stream(final_prompt)`을 직접 호출 | `StrOutputParser()`가 포함되어 있어 LLM 응답을 자동 변환 |
| **유지보수 용이성** | 체인이 없어서 복잡한 파이프라인을 만들기 어려움 | 체인을 사용하면 확장 가능성이 높고, 구성 요소를 쉽게 추가 가능 |
